# WaterTAP Introduction Tutorial
Demonstration of code-based user interface for WaterTAP

## Dependencies
* Python - Programming language
* Pyomo - Python package for equation-oriented modeling
* IDAES - Python package extending Pyomo for flowsheet modeling

## Demonstration structure 
* Pyomo example
* Property model example
* Unit model example
* Flowsheet model example
* Cost optimization example

## Pyomo Example

$${\Large
\begin{array}
 \text{\min} & x_1^2 + x_2^2  \\
 \text{s.t.} & x_1 + 2 x_2 \geq 1 
\end{array}}$$

### Solving Graphically
x1 = 0.2, x2 = 0.4, objective = 0.2

<img src="pyomo_solution.png" width="500" height="340">

### Solving with Pyomo
### Import Pyomo package and solver

In [ ]:
from pyomo.environ import ConcreteModel, Var, Reals, Objective, Constraint, value, units
from watertap.core.solvers import get_solver

### Build model

In [ ]:
# create Pyomo model
m = ConcreteModel("Pyomo Example")

# add variables
m.x1 = Var()
m.x2 = Var()

# add objective function
m.obj = Objective(expr=m.x1**2 + m.x2**2)

# add constraint
m.con = Constraint(expr=m.x1 + 2*m.x2 >= 1)

### Print model

In [ ]:
# print the model
m.pprint()

### Solve model

In [ ]:
# access the solver
solver = get_solver()

# solve the model
results = solver.solve(m)
print(results)

### Display solved model

In [ ]:
# display the model
m.display()

### Print results

In [ ]:
# print the values
print("x1 = %.2f" % value(m.x1))
print("x2 = %.2f" % value(m.x2))
print("obj = %.2f" % value(m.obj))

### Pyomo recap
* Pyomo supports equation oriented modeling and optimization capabilities within Python
* Models are created by simply specifiying variables, constraints, and objectives
* Models are solved with commercial or open-source solvers

## WaterTAP Seawater Property Example
This section shows how to create and solve a seawater property model using WaterTAP.

### Import WaterTAP and IDAES

In [ ]:
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
import watertap.property_models.seawater_prop_pack as properties

### Create a seawater state block

In [ ]:
# create pyomo model
m = ConcreteModel()

# create IDAES flowsheet
m.fs = FlowsheetBlock(dynamic=False)

# create seawater property model
m.fs.properties = properties.SeawaterParameterBlock()

# create a state block using the property model
m.fs.state_block = m.fs.properties.build_state_block([0])

# display the state block
m.fs.state_block[0].display()

### Set the state variables

In [ ]:
# fix state variables
m.fs.state_block[0].temperature.fix(273 + 25)                      # temperature (K)
m.fs.state_block[0].pressure.fix(101325)                           # pressure (Pa)
m.fs.state_block[0].flow_mass_phase_comp['Liq', 'H2O'].fix(0.965)  # mass flowrate of H2O (kg/s)
m.fs.state_block[0].flow_mass_phase_comp['Liq', 'TDS'].fix(0.035)  # mass flowrate of TDS (kg/s)

# display state block
m.fs.state_block[0].display()

### View supported properties in documentation
https://watertap.readthedocs.io/en/latest/technical_reference/property_models/seawater.html

### Create mass fraction property

In [ ]:
# attempting to access a property will automatically create the variable and constraint
m.fs.state_block[0].mass_frac_phase_comp

# display the state block
m.fs.state_block[0].display()

# note that the variable and its constraint are only created, it has not been solved yet

### Solve state block to determine mass fraction

In [ ]:
# solve the state block
solver.solve(m.fs.state_block[0])

# display the state block
m.fs.state_block[0].display()

### Create and solve osmotic pressure property

In [ ]:
# create osmotic pressure
m.fs.state_block[0].pressure_osm_phase

# solve the state block
solver.solve(m.fs.state_block[0])

# display the state block
m.fs.state_block[0].display()

# note that other intermediate variables were needed to calculate osmotic pressure
# and they were automatically created with the constraints to calculate them

### Convert osmotic pressure units to bar

In [ ]:
# convert osmotic pressure from Pa to bar
pressure_osm_bar = units.convert(m.fs.state_block[0].pressure_osm_phase["Liq"], to_units=units.bar)

# value of the osmotic pressure in bar
value(pressure_osm_bar)

### Solve for osmotic pressure using other variables

In [ ]:
# Unfix the previously fixed state variables
m.fs.state_block[0].flow_mass_phase_comp['Liq', 'H2O'].unfix()                 # mass flowrate of H2O (kg/s)
m.fs.state_block[0].flow_mass_phase_comp['Liq', 'TDS'].unfix()                 # mass flowrate of TDS (kg/s)

# Fix volumetric flowrate and TDS mass fraction
m.fs.state_block[0].flow_vol_phase['Liq'].fix(1 * (units.m ** 3 / units.hr))   # volumetric flowrate (m3/s)
m.fs.state_block[0].mass_frac_phase_comp['Liq', 'TDS'].fix(0.05)               # TDS mass fraction (-)

# Solve the state block
results = solver.solve(m.fs.state_block[0])

# Display the state block
m.fs.state_block[0].display()

# Print osmotic pressure in bar
print("Osmotic pressure: %.1f bar" % value(pressure_osm_bar))

### Solve for other variables using a specified osmotic pressure

In [ ]:
# unfix the previously fixed mass fraction
m.fs.state_block[0].mass_frac_phase_comp['Liq', 'TDS'].unfix()

# fix the osmotic pressure
m.fs.state_block[0].pressure_osm_phase["Liq"].fix(65 * units.bar)

# solve the state block
solver.solve(m.fs.state_block[0])

# display the state block
m.fs.state_block[0].display()

### Create a plot for osmotic pressure as a function of concentration

In [ ]:
# unfix osmotic pressure
m.fs.state_block[0].pressure_osm_phase["Liq"].unfix()

# simulate osmotic pressure over a range of concentrations
concentration_list = range(1, 250, 5)
pressure_osm_list = []
for c in concentration_list:
    m.fs.state_block[0].conc_mass_phase_comp["Liq", "TDS"].fix(c)  # fix concentration
    solver.solve(m.fs.state_block[0])  # solve
    pressure_osm_list.append(value(units.convert(m.fs.state_block[0].pressure_osm_phase["Liq"], to_units=units.bar))) # save osmotic pressure

In [ ]:
import matplotlib.pyplot as plt
# create figure
plt.figure(figsize=(10, 8))

# plot data
plt.plot(concentration_list, pressure_osm_list, "ko-", lw=2.0)

# format figure
plt.rc("font", size=18)
plt.xlabel("Concentration (g/L)")
plt.ylabel("Osmotic Pressure (bar)")
plt.xlim(0, 250)
plt.ylim(0, 250)

### Recap property model demonstration
* Property models create state blocks that relate state variables to the properties
* Properties are built on demand to reduce the model size
* Equation oriented modeling enables calculations in any direction
* Pyomo, IDAES, and WaterTAP support tracking and converting units of measurements

## Reverse Osmosis Demonstration
This section shows how to build, scale, initialize, and simulate reverse osmosis (RO) unit model using WaterTAP.

### Import and build RO model

In [ ]:
from watertap.unit_models.reverse_osmosis_0D import (ReverseOsmosis0D, ConcentrationPolarizationType, 
                                                     MassTransferCoefficient, PressureChangeType)

# create a Pyomo model
m = ConcreteModel()

# create IDAES flowsheet
m.fs = FlowsheetBlock(dynamic=False)

# create property model
m.fs.properties = properties.SeawaterParameterBlock()

# create RO unit model and specify options
m.fs.RO = ReverseOsmosis0D(
        property_package=m.fs.properties,
        has_pressure_change=True,
        pressure_change_type=PressureChangeType.calculated,
        mass_transfer_coefficient=MassTransferCoefficient.calculated,
        concentration_polarization_type=ConcentrationPolarizationType.calculated)

### View model structure in documentation
https://watertap.readthedocs.io/en/latest/technical_reference/unit_models/reverse_osmosis_0D.html

### Check degrees of freedom
There should be zero degrees of freedom for a simulation

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom
print("DOF = ", degrees_of_freedom(m))

### Specify values for RO variables

In [ ]:
# fix the 4 inlet state variables
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'].fix(0.035)   # feed mass flowrate of TDS (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(0.965)   # feed mass flowrate of water (kg/s)
m.fs.RO.inlet.pressure[0].fix(50 * units.bar)                    # feed pressure (Pa)
m.fs.RO.inlet.temperature[0].fix(298)                            # feed temperature (K)

# fix 2 membrane properties
m.fs.RO.A_comp.fix(4.2e-12)                                      # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)                                       # membrane salt permeability coeff (m/s)

# fix 5 module specficiations
m.fs.RO.area.fix(50)                                             # membrane stage area (m^2)
m.fs.RO.width.fix(5)                                             # membrane stage width (m)
m.fs.RO.feed_side.channel_height.fix(1 * units.mm)               # channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)                      # spacer porosity in membrane stage (-)
m.fs.RO.permeate.pressure[0].fix(101325)                         # permeate pressure (Pa)

print("DOF = ", degrees_of_freedom(m))

### Scale, initialize, and solve model

In [ ]:
# scale the model
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'TDS'))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

# initailize the model
m.fs.RO.initialize()

# solve the model
results = solver.solve(m)
print(results)

### Display report

In [ ]:
m.fs.RO.report()

### Solve RO with fixed recovery
Fix volumetric water recovery and determine the area

In [ ]:
# fix recovery and unfix area
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)
m.fs.RO.area.unfix()

# solve model
solver = get_solver()
results = solver.solve(m)

# display report
m.fs.RO.report()

## Full flowsheet optimization example
<img src="RO_ERD_flowsheet.png" width="500" height="340">

### Import and build model

In [ ]:
from pyomo.environ import Param,TransformationFactory
from pyomo.network import Arc
from idaes.core.util.scaling import constraint_scaling_transform
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Product, Feed
from idaes.core import UnitModelCostingBlock
from watertap.unit_models.pressure_changer import Pump, EnergyRecoveryDevice
from watertap.costing import WaterTAPCosting

# setup model
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = properties.SeawaterParameterBlock()
m.fs.costing = WaterTAPCosting()  # costing model

# create unit models
m.fs.feed = Feed(property_package=m.fs.properties)
m.fs.pump = Pump(property_package=m.fs.properties)
m.fs.RO = ReverseOsmosis0D(
        property_package=m.fs.properties,
        has_pressure_change=True,
        pressure_change_type=PressureChangeType.calculated,
        mass_transfer_coefficient=MassTransferCoefficient.calculated,
        concentration_polarization_type=ConcentrationPolarizationType.calculated)
m.fs.erd = EnergyRecoveryDevice(property_package=m.fs.properties)
m.fs.product = Product(property_package=m.fs.properties)
m.fs.disposal = Product(property_package=m.fs.properties)

### Connect unit models

In [ ]:
# connect unit models
m.fs.s1 = Arc(source=m.fs.feed.outlet, destination=m.fs.pump.inlet)
m.fs.s2 = Arc(source=m.fs.pump.outlet, destination=m.fs.RO.inlet)
m.fs.s3 = Arc(source=m.fs.RO.permeate, destination=m.fs.product.inlet)
m.fs.s4 = Arc(source=m.fs.RO.retentate, destination=m.fs.erd.inlet)
m.fs.s5 = Arc(source=m.fs.erd.outlet, destination=m.fs.disposal.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

### Add costing

In [ ]:
# unit equipment capital and operating costs
m.fs.pump.work_mechanical[0].setlb(0)
m.fs.pump.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
m.fs.RO.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
m.fs.erd.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing,
        costing_method_arguments={"energy_recovery_device_type": "pressure_exchanger"})

# system costing - total investment and operating costs
m.fs.costing.cost_process()
m.fs.costing.add_annual_water_production(m.fs.product.properties[0].flow_vol)
m.fs.costing.add_specific_energy_consumption(m.fs.product.properties[0].flow_vol)
m.fs.costing.add_LCOW(m.fs.product.properties[0].flow_vol)

### Scale model

In [ ]:
# scaling
# set default property values
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))

# set unit model values
set_scaling_factor(m.fs.pump.control_volume.work, 1e-3)
set_scaling_factor(m.fs.erd.control_volume.work, 1e-3)
set_scaling_factor(m.fs.RO.area, 1e-2)

# touch properties used in specifying the model
m.fs.feed.properties[0].flow_vol_phase["Liq"]
m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]
m.fs.disposal.properties[0].flow_vol_phase["Liq"]
m.fs.disposal.properties[0].mass_frac_phase_comp["Liq", "TDS"]

# calculate and propagate scaling factors
calculate_scaling_factors(m)

### Check degrees of freedom

In [ ]:
print("DOF = ", degrees_of_freedom(m))

### Specify the flowsheet

In [ ]:
# feed, 4 degrees of freedom
m.fs.feed.properties[0].flow_vol_phase["Liq"].fix(1e-3)                # volumetric flow rate (m3/s)
m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(0.035)  # TDS mass fraction (-)
m.fs.feed.properties[0].pressure.fix(101325)                           # pressure (Pa)
m.fs.feed.properties[0].temperature.fix(273.15 + 25)                   # temperature (K)

# high pressure pump, 2 degrees of freedom
m.fs.pump.efficiency_pump.fix(0.80)                                    # pump efficiency (-)
m.fs.pump.control_volume.properties_out[0].pressure.fix(75e5)          # pump outlet pressure (Pa)

# RO unit, 7 degrees of freedom
m.fs.RO.A_comp.fix(4.2e-12)                                            # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)                                             # membrane salt permeability coeff (m/s)
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)                          # volumetric recovery (-) *
m.fs.RO.feed_side.velocity[0, 0].fix(0.15)                             # crossflow velocity (m/s) *
m.fs.RO.feed_side.channel_height.fix(1e-3)                             # channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)                            # spacer porosity in membrane stage (-)
m.fs.RO.permeate.pressure[0].fix(101325)                               # permeate pressure (Pa)

# energy recovery device, 2 degrees of freedom
m.fs.erd.efficiency_pump.fix(0.80)                                     # erd efficiency (-)
m.fs.erd.control_volume.properties_out[0].pressure.fix(101325)         # ERD outlet pressure (Pa)

print("DOF = ", degrees_of_freedom(m))

### Initialize and solve the flowsheet

In [ ]:
# initialize unit by unit
solver = get_solver()

# solve feed
solver.solve(m.fs.feed)

# initialize pump
propagate_state(m.fs.s1)
m.fs.pump.initialize()

# initialize RO
propagate_state(m.fs.s2)
m.fs.RO.initialize()

# initialize energy recovery device
propagate_state(m.fs.s4)
m.fs.erd.initialize()

# propagate to product and disposal
propagate_state(m.fs.s3)
propagate_state(m.fs.s5)

# initialize cost
m.fs.costing.initialize()

# solve model
results = solver.solve(m) 
print(results)

### Display results

In [ ]:
def display_solution(m):
    print("----------system metrics----------")
    print("Recovery: %.1f %%" % (value(m.fs.RO.recovery_vol_phase[0, "Liq"])*100))
    print("Specific energy: %.2f kWh/m3" % value(m.fs.costing.specific_energy_consumption))
    print("Levelized cost of water: %.2f $/m3" % value(m.fs.costing.LCOW))

    print("\n----------inlet and outlets----------")
    print("Feed: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.feed.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Product: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.product.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.product.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Disposal: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.disposal.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.disposal.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))

    print("\n----------decision variables----------")
    print("Operating pressure: %.1f bar" %
          (value(units.convert(m.fs.pump.control_volume.properties_out[0].pressure,
                               to_units=units.bar))))
    print("Membrane area: %.1f m2" % value(m.fs.RO.area))
    print("Inlet crossflow velocity: %.1f cm/s" %
          (value(units.convert(m.fs.RO.feed_side.velocity[0, 0],
                               to_units=units.cm / units.s))))

    print("\n----------system variables----------")
    print("Pump power: %.1f kW" %
          (value(units.convert(m.fs.pump.work_mechanical[0], to_units=units.kW))))
    print("ERD power: %.1f kW" %
          (-value(units.convert(m.fs.erd.work_mechanical[0], to_units=units.kW))))
    print("Average water flux: %.1f L/(m2-h)" %
          value(units.convert(m.fs.RO.flux_mass_phase_comp_avg[0, "Liq", "H2O"]
                              / (1000 * units.kg / units.m ** 3),
                              to_units=units.mm / units.hr)))
    print("Pressure drop: %.1f bar" %
          (-value(units.convert(m.fs.RO.deltaP[0],to_units=units.bar))))
    print("Maximum interfacial salinity: %.0f ppm" %
          (value(m.fs.RO.feed_side.properties_interface[0, 1].mass_frac_phase_comp["Liq", "TDS"])*1e6))

display_solution(m)

## Setup cost optimization

In [ ]:
# objective
m.fs.objective = Objective(expr=m.fs.costing.LCOW)  # minimize the LCOW

# unfix decision variables and add bounds
# pump pressure
m.fs.pump.control_volume.properties_out[0].pressure.unfix()
m.fs.pump.control_volume.properties_out[0].pressure.setlb(10e5)
m.fs.pump.control_volume.properties_out[0].pressure.setub(85e5)
m.fs.pump.deltaP.setlb(0)

# RO crossflow velocity
m.fs.RO.feed_side.velocity[0, 0].unfix()
m.fs.RO.feed_side.velocity.setlb(0.01)
m.fs.RO.feed_side.velocity.setub(1)
m.fs.RO.area.setlb(1)
m.fs.RO.area.setub(200)

# check degrees of freedom
# operating pressure + membrane area + crossflow velocity (i.e. width) - specified water recovery
print("DOF = ", degrees_of_freedom(m))

## Solve the optimization problem and display results

In [ ]:
results = solver.solve(m)
print(results)

In [ ]:
display_solution(m)

### Change the membrane capital cost and reoptimize

In [ ]:
# double the membrane costs
m.fs.costing.reverse_osmosis.membrane_cost.fix(60)

# resolve
results = solver.solve(m)
display_solution(m)

### Summary points on cost-optimization of flowsheet models in WaterTAP
* Water treatment trains can be rapidly assembled
* WaterTAP can optimize multiple decision variables in seconds
* All parameters and variables can be modified for sensitivity analyses